In [567]:
### Import necessary library packages ###
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import math
import csv, operator
from datetime import datetime, timedelta
from cache import getCache
from rssi import rssiToDistance
from storeScale import getScale, getCoordinates, getImage, getAnchors
from storePOI import actlab_POI
import time
import pandas as pd
import json
from statistics import mean
from pytz import timezone

### Declare and instantiate variables and states ###
devlistX = []
devlistY = []
devicelist = []
tz = timezone('Asia/Singapore')
sysStartTime=tz.localize(datetime.strptime("2021-07-08 17.41.30.00", "%Y-%m-%d %H.%M.%S.%f")).timestamp()*1000#actlab_test1
sysEndTime=tz.localize(datetime.strptime("2021-07-08 17.48.05.00" , "%Y-%m-%d %H.%M.%S.%f")).timestamp()*1000
print (sysStartTime)
checkpointNum = ['A-1.1', 'A-1.2', 'A-2.1', 'A-2.2', 'A-3.1', 'A-3.2', 'A-4.1', 'A-4.2', 'A-5.1', 'A-5.2']
areaChkpts = ['A1', 'A2', 'A3', 'A4', 'A5']
checkpointTimes = {\
                    'A-1.1': datetime(2021, 7, 8, 17, 41, 37),\
                    'A-1.2': datetime(2021, 7, 8, 17, 42, 5),\
                    'A-2.1': datetime(2021, 7, 8, 17, 42, 37),\
                    'A-2.2': datetime(2021, 7, 8, 17, 43, 8),\
                    'A-3.1': datetime(2021, 7, 8, 17, 43, 46),\
                    'A-3.2': datetime(2021, 7, 8, 17, 44, 14),\
                    'A-4.1': datetime(2021, 7, 8, 17, 44, 43),\
                    'A-4.2': datetime(2021, 7, 8, 17, 45, 10),\
                    'A-5.1': datetime(2021, 7, 8, 17, 45, 38),\
                    'A-5.2': datetime(2021, 7, 8, 17, 46, 8),\
                    }

1625737290000.0


In [568]:
 ### Declare methods ###
def prepareDataByDev(devlist, startDate, endDate, devTime, dev, devX, devY):
    # This function stores the localisation data in pandas Dataframe type, resample it at every second and return it. 
    # This method can be used interchangeably for different devices (beacon, heartrate, motion sensor, etc.)
    for data in devlist:
        data[2] = float(data[2])
        data[3] = float(data[3])
        data[1] = (datetime.fromtimestamp(int(float(data[1])/1000)) + timedelta(hours=8))
        if (data[1] >= startDate) and (data[1] <= endDate):
            devTime.append(data[1])
            dev.append(data[0])
            devX.append(data[2])
            devY.append(data[3])
    devData = pd.DataFrame({'timestamp': devTime, 'Device': dev, 'Beacon Longtitude': devX, 'Beacon Latitude': devY})
    devData.set_index('timestamp', inplace=True)
    devData.index = pd.to_datetime(devData.index)
    devData = devData.groupby('Device').resample('S')['Beacon Longtitude', 'Beacon Latitude'].mean().ffill()
    return devData

def prepareData(datalist):
    # This function prepares the data and returns it in pandas dataFrame type, as well as returning the device(s) ID.
    devTime, dev, devX, devY = [], [], [], []
    initialDate = datetime.fromtimestamp(int(float(sysStartTime)/1000)) + timedelta(hours=8)
    finalDate = datetime.fromtimestamp(int(float(sysEndTime)/1000)) + timedelta(hours=8)
    devData = prepareDataByDev(datalist, initialDate, finalDate, devTime, dev, devX, devY)
    findDevicelist = devData.index.values.tolist()
    for findDev in findDevicelist:
        if findDev[0] not in devicelist:
            devicelist.append(findDev[0])
    return devData, devicelist

def prepareFilteredData(rawData):
    # This function stores the raw RSSI data into an array
    dev, devTime, devRSSI = [],[],[]
    rawData = sorted(rawData, key=operator.itemgetter(2), reverse=False)
    for data in rawData:
        if int(data[2]) >= int(sysStartTime) and int(data[2]) <= int(sysEndTime):
            devTime.append(data[2])
            dev.append(data[1])
            devRSSI.append(float(data[3]))
    devData = pd.DataFrame({'timestamp': devTime, 'Anchor': dev, 'RSSI': devRSSI})
    devData = devData.groupby('Anchor').agg(list)
    return devData

def filterRSSI(rssiDict, alpha):
    # This functions filters the raw RSSI data using filter with
    # a particular alpha gain
    filteredData={}
    rssilist = rssiDict.RSSI
    timestamplist = rssiDict.timestamp
    mu = int(rssilist[0])
    time = int(timestamplist[0])
    sigma=1000
    period=100
    lastUpdate=time
    muList=[]
    sigmaList=[]
    alp = alpha
    j=0
    while j< len(rssilist):
        diff= int(rssilist[j]) - mu
        incr = alp * diff 
        mu += incr
        sigma = (1-alp) * (sigma+diff *incr)

        diff = int(timestamplist[j]) - lastUpdate - period
        incr = 0.01*diff

        period += incr
        lastUpdate = int(timestamplist[j])

        if period >100:
            alp=alpha
        else:
            alp = period*0.0001667 + 0.0333

        muList.append(mu)
        sigmaList.append(sigma)
        j+=1

    try:
        filteredData['rssi'].update(muList)
    except KeyError:
        filteredData = {'rssi': muList}

    return filteredData

def convertDevice(receiverId):
    return cache_anchors[receiverId]['device']['id']

def shiftTimeAxis(timestampList):
    newTimeList=[]
    i=0
    while i< len(timestampList):
        newTimeList.append((int(timestampList[i])-sysStartTime)/1000)
        i+=1

    return newTimeList

def read_file(fileName):
    # This function reads the location file and returns the datalist
    global finallist
    with open(fileName, 'r') as f:
        next(f)
        reader = csv.reader(f)
        devlist = list(reader)
    return devlist

def updateChkpt_timings(_checkpoints, checkpointNum, checkpointTimes):
    # This function updates the _checkpoints dictionary with the start and end time of each checkpoints and return it
    for checkpointID in checkpointNum:
        _checkpoints[checkpointID].update({'start': checkpointTimes[checkpointID], 'end': checkpointTimes[checkpointID] + timedelta(seconds=20)})
    return _checkpoints

def categoriseData(_beacon, _checkpoints):
    # This function breaks down the location dataFrame into their corresponding checkpoints,
    # catching those location that falls in the checkpoints timings
    temp = {}
    timestampList = _beacon.loc['b2',:].index.tolist();
    beaconX = _beacon.loc['b2', 'Beacon Longtitude'].tolist();
    beaconY = _beacon.loc['b2', 'Beacon Latitude'].tolist();
    for k,v in _checkpoints.items():
        for index, beaconTime in enumerate(timestampList):
            if (beaconTime >= v['start'] and beaconTime <= v['end']):
                try:
                    temp[k].append([{'lng': beaconX[index], 'lat': beaconY[index], 'timestamp': tz.localize(beaconTime).timestamp()*1000}])
                except KeyError:
                    temp[k] = [[{'lng': beaconX[index], 'lat': beaconY[index], 'timestamp': tz.localize(beaconTime).timestamp()*1000}]]
    return temp

def calculateBeaconErr(_beacon, _checkpoints, mapScale):
    # This function calculates the mean error distance, standard deviation and standard error for each checkpoint and return them in a dictionary
    beaconMeanVariance = {}
    errorDist = []
    checkptlist = []
    timestamplist = []
    errorDict = {}
    temp = []
    errorDict = {key : {'error': [], 'checkpt': [], 'timestamp': []} for key in _beacon.keys()}
    for k,v in _beacon.items():
        errorDist = [abs(math.sqrt((_checkpoints[k]['lng']-loc[0]['lng'])**2 + (_checkpoints[k]['lat']-loc[0]['lat'])**2))*mapScale for loc in v]
        checkptlist = [k for loc in v]
        timestamplist = [(loc[0]['timestamp'] - sysStartTime)/1000 for loc in v]
        errorDict[k]['error'] += errorDist
        errorDict[k]['checkpt'] += checkptlist
        errorDict[k]['timestamp'] += timestamplist
        errorDist = []
        checkptlist = []
        timestamplist = []
    df = pd.DataFrame.from_dict(errorDict)
    return df

def sortResults(_result):
    # This function sort the results in the proper order as defined in areaChkpts and return it
    temp = {}
    for num in checkpointNum:
        try:
            temp[num].update(_result[num])
        except KeyError:
            temp[num] = _result[num]
    _result = temp
    return _result

In [569]:
### continue - declare Methods ###
def findMinimum_Maximum(_list):
    # This function is used for the map configuration to determine the minimum and maximum lat and lng points for a given map coordinates.
    minimum = float("{0:.3f}".format(float(_list[0])))
    maximum = float("{0:.3f}".format(float(_list[0])))
    for a in range(1, len(_list)):
        temp = float("{0:.3f}".format(float(_list[a])))
        if temp<minimum:
            minimum = temp
        elif temp>maximum:  
            maximum = temp
    return minimum, maximum


def getMap(mapName):
    # This function returns all the necessary points for plotting the map
    coord_lng_list = []
    coord_lat_list = []
    coord_total_list = []
    map_coordinates = getCoordinates(mapName)
    for coor in map_coordinates:
        coord_lng_list.append(coor[0])
        coord_lat_list.append(coor[1])
    coord_total_list = coord_lng_list + coord_lat_list
    min_pt, max_pt = findMinimum_Maximum(coord_total_list)
    min_x, max_x = findMinimum_Maximum(coord_lng_list)
    min_y, max_y = findMinimum_Maximum(coord_lat_list)
    return min_pt, max_pt, min_x, max_x, min_y, max_y

def showMap(result):
    image = go.Figure()
    meanValues = []
    imageURL = getImage('actlab')
    img_width = 1600
    img_height = 900
    meanValues = [mean(result[k]['error']) for k in result.keys()]
    image.add_layout_image(
            x=0,
            sizex=img_width,
            y=0,
            sizey=img_height,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            source=imageURL
    )

    # Line shape added programatically
    image.add_shape(type='rect', xref='x', yref='y', x0=100, x1=250, y0=150, y1=300, line_color='darkblue')
    image.add_annotation(x=180, y=220, text="A1.1", showarrow=False, font=dict(size=14,color="black"))
    image.add_shape(type='rect', xref='x', yref='y', x0=100, x1=250, y0=300, y1=440, line_color='darkblue')
    image.add_annotation(x=180, y=360, text="A1.2", showarrow=False, font=dict(size=14,color="black"))
    image.add_shape(type='rect', xref='x', yref='y', x0=50, x1=190, y0=440, y1=500, line_color='red')
    image.add_annotation(x=120, y=470, text="A2.1", showarrow=False, font=dict(size=14,color="black"))
    image.add_shape(type='rect', xref='x', yref='y', x0=190, x1=330, y0=440, y1=500, line_color='red')
    image.add_annotation(x=260, y=470, text="A2.2", showarrow=False, font=dict(size=14,color="black"))
    image.add_shape(type='rect', xref='x', yref='y', x0=330, x1=430, y0=150, y1=300, line_color='brown')
    image.add_annotation(x=380, y=220, text="A3.1", showarrow=False, font=dict(size=14, color="black"))
    image.add_shape(type='rect', xref='x', yref='y', x0=330, x1=430, y0=300, y1=440, line_color='brown')
    image.add_annotation(x=380, y=360, text="A3.2", showarrow=False, font=dict(size=14, color="black"))
    image.add_shape(type='rect', xref='x', yref='y', x0=330, x1=440, y0=440, y1=500, line_color='green')
    image.add_annotation(x=380, y=470, text="A4.1", showarrow=False, font=dict(size=14, color="black"))
    image.add_shape(type='rect', xref='x', yref='y', x0=440, x1=550, y0=440, y1=500, line_color='green')
    image.add_annotation(x=490, y=470, text="A4.2", showarrow=False, font=dict(size=14, color="black"))
    image.add_shape(type='rect', xref='x', yref='y', x0=420, x1=550, y0=500, y1=650, line_color='violet')
    image.add_annotation(x=490, y=570, text="A5.1", showarrow=False, font=dict(size=14, color="black"))
    image.add_shape(type='rect', xref='x', yref='y', x0=420, x1=550, y0=650, y1=800, line_color='violet')
    image.add_annotation(x=490, y=720, text="A5.2", showarrow=False, font=dict(size=14, color="black"))
    
    image.update_layout(dragmode='drawrect', newshape=dict(line_color='cyan'), title_text='Actlab Map Image with Mean Error (w/o calibration)', 
        xaxis_visible = False, yaxis_visible = False)
    image.update_xaxes(showgrid=False, range=(0, img_width))
    image.update_yaxes(showgrid=False, scaleanchor='x', range=(img_height, 0))
    image.add_trace(go.Scatter(
        x=[180, 180, 120, 260, 380, 380, 380, 490, 490, 490], y=[220, 360, 470, 470, 220, 360, 470, 470, 570, 720], mode='markers', customdata=[err for err in meanValues],
        text=[area for area in areaChkpts], marker=dict(color=[err for err in meanValues], colorscale="YlOrRd",
            showscale=True, size=[35 for err in meanValues], cmin=1, cmax=3, 
            colorbar=dict(title="Mean Error", tickvals=[1, 2, 3])),
            hovertemplate=
                "Mean Error: %{customdata:.1f}<br>" +
                "<extra></extra>"
            ))
    return image

def createFigure(legend_name):
    fig = go.Figure()
    fig.update_layout(legend=dict(orientation="v", xanchor="left"), barmode='group')
    fig.update_layout(legend_title_text=legend_name)
    fig.update_layout(title="Violin Plot of Error Distances")
    fig.update_xaxes(title_text="Checkpoints")
    fig.update_yaxes(range=[-1.5,7.5])
    return fig

def createFigureForDistances(legend_name):
    fig = go.Figure()
    fig.update_layout(legend_title_text=legend_name)
    fig.update_layout(title="Error Distances Over Time")
    fig.update_xaxes(title_text="Time")
    fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 20
    )
)
    return fig

def createFigureForFilteredData():
    fig = make_subplots(rows=1, cols=3, column_widths=[0.5, 0.5, 0.5], 
        subplot_titles=("Normal Filtering (alpha=0.2)", "Decreased Filtering (alpha=0.6)", 
                        "Increased Filtering (alpha=0.05)"), shared_yaxes=True, shared_xaxes=True)
    fig.update_xaxes(title_text="Time from Experiment Start Time (secs)", row=1, col=2)
    fig.update_yaxes(title_text="RSSI (dbM)", row=1, col=1)
    fig.update_layout(legend=dict(orientation="v", xanchor="left"), barmode='group')
    fig.update_layout(hovermode="x unified")
    fig.update_xaxes(showspikes=True, spikecolor="purple", spikesnap="cursor", spikemode="across", spikethickness=2)
    fig.update_yaxes(showspikes=True, spikecolor="orange", spikethickness=2)
    return fig

In [570]:
def plotValues(fig, result, fileName, mode):
    name = 'Mean Error'
    stdName='Standard Deviation'
    for area in checkpointNum:
        fig.add_trace(go.Violin(
            name=area,
            x=result[area]['checkpt'], y=result[area]['error'],
            meanline_visible=True,
            box_visible=True
        ))
    fig.update_traces(meanline_visible=True,
                  points='all', # show all points
                  jitter=0.05,  # add some jitter on points for better visibility
                  scalemode='count') #scale violin plot area with total count
    return fig

def plotValuesForDistance(fig, result):
    for area in checkpointNum:
        fig.add_trace(go.Scatter(
            name=area, x=result[area]['timestamp'], y=result[area]['error'], mode='lines+markers'))
    return fig

def plotValuesForFilteredData(fig, result, anchorName):
    timeAxis=shiftTimeAxis(result.loc[anchorName].timestamp)
    fig.add_trace(go.Scatter(
        name="Raw RSSI", x=timeAxis, y=result.loc[anchorName].RSSI, mode='lines', opacity=.5), row=1, col=1)
    fig.add_trace(go.Scatter(
        name="Raw RSSI", x=timeAxis, y=result.loc[anchorName].RSSI, mode='lines', opacity=.5), row=1, col=2)
    fig.add_trace(go.Scatter(
        name="Raw RSSI", x=timeAxis, y=result.loc[anchorName].RSSI, mode='lines', opacity=.5), row=1, col=3)
    rssi_filtered = filterRSSI(result.loc[anchorName], 0.2)
    fig.add_trace(go.Scatter(name="Filtered RSSI", x=timeAxis, y=rssi_filtered['rssi'], mode='lines', line_color="black"), row=1, col=1)
    rssi_filtered = filterRSSI(result.loc[anchorName], 0.6)
    fig.add_trace(go.Scatter(name="Filtered RSSI", x=timeAxis, y=rssi_filtered['rssi'], mode='lines', line_color="black"), row=1, col=2)
    rssi_filtered = filterRSSI(result.loc[anchorName], 0.05)
    fig.add_trace(go.Scatter(name="Filtered RSSI", x=timeAxis, y=rssi_filtered['rssi'], mode='lines', line_color="black"), row=1, col=3)
    return fig

In [571]:
### Run Filter Program ###
def filter_visualise(fileName):
    devlist = read_file(fileName)
    RSSI_raw = prepareFilteredData(devlist)
    return RSSI_raw

In [572]:
### Main program ###
def main(fileName):
    # Steps to run this whole program is: 
    # Getting list of checkpoints and map scale from server
    # Update the checkpoint list with its timings
    # Export data into a list from the location file
    # Getting map configuration
    # Categorise Data into their respective checkpoints
    # Calculate their error distance, standard deviation and standard error
    # Sort the results
    checkpointslist = actlab_POI()
    mapScale = getScale('actlab')
    checkpointslist = updateChkpt_timings(checkpointslist, checkpointNum, checkpointTimes)
    devlist = read_file(fileName)
    finallist, devicelist = prepareData(devlist)
    dataInCategory = categoriseData(finallist, checkpointslist)
    result = calculateBeaconErr(dataInCategory, checkpointslist, mapScale)
    result = sortResults(result)
    return result, checkpointslist, fileName

In [573]:
### Below are execution of the main program to read and run from different files (normal filter, increased filter, calibration etc.) in order to see the different effects
### Analysis data for normal filter ###
fig1 = createFigure('Normal Filter')
fig_dist = createFigureForDistances('Normal Filter')
filtered_fig = createFigureForFilteredData()
result, checkpointslist, fileName = main('./Filter_0.2/actlab_location_test1_08Jul.csv')
RSSI_raw = filter_visualise('./raw-data/actlab_loc_08Jul.csv')
fig1 = plotValues(fig1, result, fileName, 'mean')
fig_dist = plotValuesForDistance(fig_dist, result)
filtered_fig = plotValuesForFilteredData(filtered_fig, RSSI_raw, 'b827ebed958b')
image = showMap(result)
image.show(config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})
# filtered_fig.show()
# fig1.show()
# fig_dist.show()

In [574]:
filtered_fig.show()
fig1.show()
fig_dist.show()

In [575]:
### Analysis data for no filter ###
fig = createFigure('No Filter')
fig_dist = createFigureForDistances('No Filter')
result, checkpointslist, fileName = main('./noFilter/actlab_location_test1_08Jul.csv')
fig = plotValues(fig, result, fileName, 'mean')
fig_dist = plotValuesForDistance(fig_dist, result)
fig.show()
fig_dist.show()

In [576]:
### Analysis data for decreased filter with alpha gain of 0.6 ###
fig2 = createFigure('Decreased Filter')
fig_dist = createFigureForDistances('Decreased Filter')
result, checkpointslist, fileName = main('./Filter_0.6/actlab_location_test1_08Jul.csv')
fig2 = plotValues(fig2, result, fileName, 'mean')
fig_dist = plotValuesForDistance(fig_dist, result)
fig2.show()
fig_dist.show()

In [577]:
### Analysis data for increased filter with alpha gain of 0.05 ###
fig3 = createFigure('Increased Filter')
fig_dist = createFigureForDistances('Increased Filter')
result, checkpointslist, fileName = main('./Filter_0.05/actlab_location_test1_08Jul.csv')
fig3 = plotValues(fig3, result, fileName, 'mean')
fig_dist = plotValuesForDistance(fig_dist, result)
fig3.show()
fig_dist.show()

In [578]:
### Analysis data for calibrated Measured Power with normal filter ###
fig4 = createFigure('Calibration')
result, checkpointslist, fileName = main('./Filter_0.2/anchorCalibration/actlab_location_test1_08Jul.csv')
fig4 = plotValues(fig4, result, fileName, 'calibrate')
fig4.show()